In [1]:
import constant as c
import functions as f
import pandas as pd
import os
from os.path import join
from itertools import combinations
import numpy as np
from scipy.spatial.distance import euclidean, hamming
import collections

from numpy import matlib

import proxmin


In [2]:
ant_dataframe = []

for filename in os.listdir(c.ant):
    tmp = pd.read_csv(join(c.ant, filename))
    ant_dataframe.append(tmp)

ant_dataframe = pd.concat(ant_dataframe)

In [3]:
ant_dataframe

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,11,4,2,14,42,29,2,...,1.000000,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,14,1,1,8,32,49,4,...,1.000000,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,3,2,0,1,9,0,0,...,1.000000,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,12,3,0,12,37,32,0,...,1.000000,1,0.770833,0.458333,0,0,24.083333,3,1.4167,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,6,3,0,4,21,1,0,...,1.000000,0,0.880952,0.416667,2,2,21.000000,1,0.8333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,ant,1.7,org.apache.tools.ant.taskdefs.Javadoc,92,3,0,34,261,3726,8,...,0.970588,11,0.291339,0.112476,2,2,28.021739,15,1.5543,4
741,ant,1.7,org.apache.tools.ant.types.selectors.BaseSelector,6,3,6,10,10,3,7,...,1.000000,0,0.857143,0.500000,0,0,6.500000,3,1.5000,0
742,ant,1.7,org.apache.tools.ant.types.resources.selectors...,7,3,5,9,26,0,5,...,1.000000,0,0.857143,0.314286,1,3,19.000000,3,1.5714,0
743,ant,1.7,org.apache.tools.ant.taskdefs.compilers.Gcj,5,2,0,8,34,8,1,...,1.000000,0,0.884615,1.000000,0,0,42.000000,11,3.4000,1


In [4]:
needed = ['name', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']

In [5]:
versions = ant_dataframe.version.unique()
names = ant_dataframe.name.unique()
version_tuple = [(x, y) for x, y in zip(versions[0::1],versions[1::1])]

In [6]:
for v in version_tuple:
    previous = ant_dataframe.loc[ant_dataframe.version == v[0]]
    current = ant_dataframe.loc[ant_dataframe.version == v[1]]
    previous = previous.sort_values('name').reset_index()
    current = current.sort_values('name').reset_index()
    previous = previous.drop(['index'], axis = 1)
    current = current.drop(['index'], axis = 1)
    modules_previous = previous.name.unique()
    modules_current = current.name.unique()
    D = f.create_D(current, previous, features, 'euclidean')
    #Z = f.create_Z(current, previous, D)
    #Z = np.random.randint(2, size = (n, d))

    break

In [7]:
dis = pd.DataFrame(D)
dis

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,852.838871,622.123658,595.268394,711.701084,716.314891,721.424934,436.609358,601.045972,657.290126,784.330848,...,444.729841,343.075666,341.582207,714.486484,641.656021,273.263023,713.288166,574.367228,448.643799,631.867980
1,1415.367161,6.480741,192.378977,91.494134,94.866471,100.970653,186.608634,25.161683,47.969638,1404.417831,...,179.894653,304.198619,283.937850,94.149205,28.480713,405.719347,92.144557,54.525767,1066.048380,19.110177
2,1375.107580,137.882816,52.364110,194.680159,202.925965,210.024562,187.581421,142.992752,169.799341,1354.034998,...,187.900507,292.556913,265.779356,204.022834,160.623427,384.822171,200.583978,138.842111,1018.323009,147.400461
3,1494.856149,91.581530,236.922697,7.071068,16.408839,26.495283,275.425505,113.228944,68.553710,1493.361285,...,269.400038,387.179071,374.248577,25.159491,70.609992,486.916431,6.504509,143.103941,1156.503903,85.546622
4,1503.890734,95.507316,241.691844,19.729420,1.414214,10.781929,280.705836,116.040514,67.694370,1498.241792,...,273.252778,394.945777,377.817964,10.062306,77.513844,495.372589,10.247101,144.676642,1160.079212,86.434386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1488.564734,78.949812,232.401198,32.588341,22.366269,24.041631,263.080859,97.926034,51.477611,1479.441342,...,254.501534,378.994065,358.955193,17.000000,63.958745,480.057131,26.176278,125.345218,1140.735579,67.787031
121,729.890244,692.085653,669.472455,775.848348,783.501813,789.767907,508.977424,673.063202,729.189198,747.167673,...,523.197647,388.919871,433.682357,783.138779,706.414292,292.123998,778.636025,650.718967,462.354492,704.625808
122,937.065928,513.623255,497.027634,602.622383,607.496731,612.680949,327.888791,492.667806,548.954746,892.062077,...,336.097331,233.550631,236.005193,605.776316,532.306708,175.482578,604.246025,466.283748,559.136499,523.611360
123,1100.758944,342.716459,344.115013,432.688218,436.794752,441.831774,158.700024,321.464953,378.032687,1063.036693,...,165.673777,108.743157,66.620521,434.902652,362.616553,146.158769,433.954649,294.639930,724.440220,352.194897


In [8]:
D.shape

(125, 178)

In [9]:
def augmentD(D, beta, sigmaV = 1):
    dmax = np.max(np.amax(D, axis = 0)) + 1
    Ciccio = np.empty([len(previous), len(current)])
    for index, d in enumerate(D):
        k = [1 if c == 0 else 0 for c in d]
        Ciccio[index] = k
    w = np.multiply(beta, np.divide(np.exp(-np.min(D + np.multiply(dmax, Ciccio))), sigmaV))
    D = np.concatenate((D,w), axis=None)
    return D
    


In [10]:
def computRegularizer(D, q):
    Nr = range(D.shape[0])
    Nc = range(D.shape[1])
    idx = np.argmin(np.sum(D, axis=1))
    rho_max = float('-inf');
    idxC = np.setdiff1d(Nr, idx)
    
    if q == 2:
        for i in range(D.shape[0]):
            v = D[idxC[0]] - D[idx]
            p = np.sqrt(D.shape[0]) * np.linalg.norm(v, ord=2)**2 / (2* sum(v))
            if(p > rho_max):
                rho_max = p
    elif q == float('inf'):
        for i in range(D.shape[0]):
            v = D[idxC[0]] - D[idx]
            p = np.linalg.norm(v, ord=1)/ 2
            if(p > rho_max):
                rho_max = p
                
    if D.shape[0] == D.shape[1]:
        a = np.zeros((D.shape[0], D.shape[1] ), float)
        np.fill_diagonal(a, 10**10)
        rho_min = np.min(D + a)
    else:
        rho_min = np.min(D)
    
    return rho_min, rho_max

    

In [11]:
def ds3solver_constrained(D, p, rho, mu, maxIter, errThr, cfd):
    ratio = 0.1
    CFD = cfd
    
    Nr = range(D.shape[0])
    Nc = range(D.shape[1])
    
    terminate = False
    k = 1

    idx = np.argmin(np.sum(D, axis=1))
    
    C1 = np.zeros(D.shape)
    C1[idx,:] = 1
    
    Lambda = np.zeros(D.shape);

    while(not terminate):
        Z  = solver_BCLS_closedForm(C1 - np.divide((Lambda + D), mu))
        C2 = projL1Inf(Z+Lambda/mu, tau, CFD)
        
        Lambda = Lambda + np.multiply(mu, (Z - C2))
        
        err1 = errorCoef(Z,C2)
        err2 = errorCoef(C1,C2)   
        
        if (k >= maxIter or (err1 <= errThr and err2 <= errThr)):
            terminate = True
            print('Terminating:')
            #print('||Z-C||= %1.2e, ||C1-C2||= %1.2e, repNum = %3.0f, iteration = %.0f \n\n',err1,err2,len(findRepresentatives(C2)),k)
        else:
            k = k +1
            #if(np.mod(k, 100) == 0): 
                #print('||Z-C||= %1.2e, ||C1-C2||= %1.2e, repNum = %3.0f, iteration = %.0f \n\n',err1,err2,len(findRepresentatives(C2)),k)
        C1 = C2
    return C2

In [12]:
def errorCoef(Z,C):
    return np.sum(np.sum(np.abs(Z-C) )) / Z.shape[0] * Z.shape[1]

In [13]:
def ds3solver_regularized(D, p, rho, mu, maxIter, errThr, cfd):
    ratio = 0.1
    CFD = cfd
    
    Nr = range(D.shape[0])
    Nc = range(D.shape[1])
    
    terminate = False
    k = 1

    idx = np.argmin(np.sum(D, axis=1))
    
    C1 = np.zeros(D.shape)
    C1[idx,:] = 1
    
    Lambda = np.zeros(D.shape);

    while(not terminate):
        Z  = shrinkL1Lp(np.divide(C1 - (Lambda +  D), mu) , rho/mu*CFD, p)
        C2 = solver_BCLS_closedForm(np.divide((Z + Lambda), mu))
        
        Lambda = Lambda + np.multiply(mu, (Z - C2))
        
        err1 = errorCoef(Z,C2)
        err2 = errorCoef(C1,C2)   
        
        if (k >= maxIter or (err1 <= errThr and err2 <= errThr)):
            terminate = True
            print('Terminating:')
            #print('||Z-C||= %1.2e, ||C1-C2||= %1.2e, repNum = %3.0f, iteration = %.0f \n\n',err1,err2,len(findRepresentatives(C2)),k)
        else:
            k = k +1
            #if(np.mod(k, 100) == 0): 
                #print('||Z-C||= %1.2e, ||C1-C2||= %1.2e, repNum = %3.0f, iteration = %.0f \n\n',err1,err2,len(findRepresentatives(C2)),k)
        C1 = C2
    return C2

In [14]:
def L2_Linf_shrink(y,t):
    x = y
    o = np.sort(np.abs(y))[::-1]
    z = y[o]
    
    mz = np.abs(z)
    #cumsum(abs(z(1:length(z)-1)))./(1:length(z)-1)'-t./(1:length(z)-1)';
    
    cs = np.divide(np.abs(z[:len(z)-1]).cumsum(axis=0), np.divide( np.array(range(len(z) -1)).T - t,  np.array(range(len(z) -1)).T))

    d = (cs > np.abs(z[1: len(z)])).choose(np.abs(z[1: len(z)]))
    
    if (np.sum(d) == 0):
        cut_index = len(y)
    else:
        cut_index = np.where(d == 1)
    
    
    zbar = np.mean(np.abs(z[:cut_index]))
    
    if (cut_index < len(y)):
        x[o[:cut_index]] = np.sign(z[:cut_index])*np.max(zbar-t/cut_index, np.abs(z[cut_index+1:]))
    else:
        x[o[:cut_index]] = np.sign(z[:cut_index])*np.max(zbar-t/cut_index, 0)
    
    return x

In [15]:
def shrinkL1Lp(C1, Lambda, p):
    C2 = []
    if Lambda.size != 0:
        D = C1.shape[0]
        N = C1.shape[1] 
        if(p == 1):
            C2 = np.multiply(np.max(np.abs(C1) - np.matlib.repmat(Lambda,1,N), 0), np.sign(C1))
        elif (p == 2):
            r = np.max(np.sqrt(np.sum(C1**2, 1)) - Lambda, 0)
            C2 = np.multiply(np.matlib.repmat(np.divide(r, (r+Lambda)), 1, N), C1)
        elif (p == float('inf')):
            C2 = np.zeros(D, N)
            for j in range(len(D)):
                C2[j, :] = L2_Linf_shrink(C1[j, :].T, Lambda[j].T)
    return C2

In [16]:
def run_ds3(D, p = float('Inf'), regularized = False, alpha = 3, outlierDetect = False,
           beta = 1):
    D = np.divide(D, np.max(np.amax(D, axis = 0)))
    
    CFD = np.ones((D.shape[1], 1))
    if (outlierDetect):
        D = augmentD(D,beta)
        CFD = np.concatenate((D,np.spacing(1)), axis=None)
    
    if(regularized):
        rho_min, rho_max = computRegularizer(D, p)
        rho = alpha * rho_max
        mu = 1 * 10**-1
        maxIter = 3000
        errThr = 1 * 10**-7
        cfd = CFD
        Z = ds3solver_regularized(D, p, rho, mu, maxIter, errThr, cfd)
    else:
        rho = alpha
        mu = 1 * 10**-1
        maxIter = 3000
        errThr = 1 * 10**-7
        cfd = CFD
        Z = ds3solver_constrained(D, p, rho, mu, maxIter, errThr, cfd)
        
    

In [56]:
from smop.libsmop import *

In [83]:
range(1,1 - 1)

range(1, 0)

In [110]:
def solver_BCLS_closedForm(U):
    m = U.shape[0]
    N = U.shape[1]
    
    V = np.sort(U)[::1]
    
    activeSet = np.array(range(N))
    
    theta = np.zeros((1, N))
    
    i = 1
    
    print(V[i, activeSet] <= 0)
    
    print(theta[V[i, activeSet] <= 0])
    return
    while(activeSet.size != 0 and i <= m):
        idx = np.where((V[i,activeSet] - (np.sum(V[:i, activeSet], 1) -1)/ i) <= 0)
        theta[activeSet[idx]] = (np.sum(V[:i-1, activeSet[idx]], 1) - 1) / (i-1)
        
        activeSet[idx] = []
        
        i += 1
    if(activeSet.size != 0):
        theta[activeSet] = (np.sum(V[:m, activeSet],1) -1) /m
    
    C = np.max(U- np.matlib.repmat(theta,m,1), 0)
    
    return C

In [111]:
run_ds3(D)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 178